In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
data_location = os.path.dirname(os.path.dirname(os.path.abspath('__file__'))) + "\\Data"

## Load data


In [ ]:
train_df = pickle.load(open(data_location + '/train_df', 'rb'))
test_df = pickle.load(open(data_location + '/test_df', 'rb'))

In [ ]:
#super_df = pd.concat([train_df,test_df],ignore_index=True)
#X_train, X_test, y_train, y_test = train_test_split(super_df.drop(columns = ["RUL", "unit"], axis=1), super_df["RUL"], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns = ["RUL", "unit"], axis=1), train_df["RUL"], test_size=0.1, random_state=42)

In [ ]:
X_train = X_train[0:100]
y_train = y_train[0:100]

# Benchmark 

In [ ]:
means = [y_train.mean()] * y_train.size
print(mse(means, y_train))

# Linear

In [ ]:
from sklearn.linear_model import LinearRegression

LM = LinearRegression()
LM.fit(X_train, y_train)
#cross_validate(model, X_train, y_train)
#print("Train mae: "+ str(mae(model.predict(x_train) , y_train))))
#print("Validation mae: "+ str(mae(model.predict(x_val) , y_val))))

# Tree regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
DT = DecisionTreeRegressor(random_state = 42)

DT_random_grid = {'min_samples_split': range(2, 10),
               'min_samples_leaf': range(1, 5),
               'max_features': ["auto", "sqrt", "log2"],}

DT_gs  = RandomizedSearchCV(estimator = DT, n_jobs=-1, scoring = "neg_mean_squared_error",
                        param_distributions=DT_random_grid,n_iter=80,cv=5,iid=True,return_train_score =True)

DT_gs.fit(X_train,y_train)

DT = DT_gs.best_estimator_
print(DT_gs.best_score_) 
print(DT_gs.best_params_)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RF = model = RandomForestRegressor(criterion="mse", random_state = 42, verbose = 1)


RF_random_grid = {'n_estimators': range(10, 300),
               'max_features': ['auto', 'sqrt', 'log2'],
               'min_samples_split': range(2, 10),
               'min_samples_leaf': range(1, 5)}

RF_gs  = RandomizedSearchCV(estimator = RF, n_jobs=-1, scoring = "neg_mean_squared_error",
                        param_distributions=RF_random_grid,n_iter=80,cv=5,iid=True,return_train_score =True, verbose = 1)

RF_gs.fit(X_train,y_train)

RF = RF_gs.best_estimator_
print(RF_gs.best_score_) 
print(RF_gs.best_params_)

# Boosted tree regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
GB =  GradientBoostingRegressor(random_state = 42)


GB_random_grid = {'n_estimators': range(10, 300),
               'learning_rate': [0.01, 0.05, 0.1, 0.2],
               'min_samples_split': range(2, 10),
               'min_samples_leaf': range(1, 5),
                 'max_depth': range(2,8)}

GB_gs  = RandomizedSearchCV(estimator = GB, n_jobs=-1, scoring = "neg_mean_squared_error",
                        param_distributions=GB_random_grid,n_iter=82,cv=5,iid=True,return_train_score =True, verbose = 1)

GB_gs.fit(X_train,y_train)

GB = GB_gs.best_estimator_
print(GB_gs.best_score_) 
print(GB_gs.best_params_)

# Comparison

In [ ]:
models = {"Linear regression" : {"model" : LM}, "Tree regressor" : {"model" : DT}, "Random forest regressor" : {"model" : RF}, "Gradient Boosting Regressor" : {"model" : GB}}

In [ ]:
print(mse([y_train.mean()]*len(y_train), y_train))
print(mse([y_train.mean()]*len(y_test), y_test))


In [ ]:
for key in models.keys():
    models[key]["train_mse"] = mse(models[key]["model"].predict(X_train), y_train)
    models[key]["test_mse"] = mse(models[key]["model"].predict(X_test), y_test)

In [ ]:
model_names = list(models.keys())
train_mse = [models[model]["train_mse"] for model in model_names]
test_mse = [models[model]["test_mse"] for model in model_names]

In [ ]:
model_summary = pd.DataFrame({"Models" : model_names, "MSE training set" : train_mse, "MSE test set" : test_mse}).round(2)

In [ ]:
model_summary

In [ ]:
model_summary.to_excel("model_summary.xlsx")

In [ ]:
joblib.dump(RF, 'RF.joblib')

# Final model

In [ ]:
# set parameters

params = GB_gs.best_params_

lower = GradientBoostingRegressor().set_params(**params)
mid = GradientBoostingRegressor().set_params(**params)
upper = GradientBoostingRegressor().set_params(**params)

LOWER_ALPHA = 0.025

UPPER_ALPHA = 0.975

lower.loss = "quantile"
lower.alpha = LOWER_ALPHA
upper.loss = "quantile"
upper.alpha = UPPER_ALPHA

In [ ]:
# Fit models
lower.fit(X_train, y_train)
mid.fit(X_train, y_train)
upper.fit(X_train, y_train)

In [ ]:
#Save models
joblib.dump(lower,'gb_lower_model.joblib')
joblib.dump(mid, 'gb_model.joblib')
joblib.dump(upper, 'gb_upper_model.joblib')

# Prediction interval comparison

In [ ]:
RF = joblib.load("RF.joblib")

In [ ]:
def pred_ints(model, X, percentile=95):
    err_down = []
    err_up = []
    y = []
    for x in X:
        x = x.reshape(1, -1)
        preds = []
        for pred in model.estimators_:
            preds.append(pred.predict(x))
        err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
        err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
#        y.append(model.predict(x))
    return err_down, y, err_up

In [ ]:
low_RF, y, up_RF = pred_ints(RF, X_test.values)

In [ ]:
low_GB, up_GB = lower.predict(X_test), upper.predict(X_test)

In [ ]:
truth = y_test
correct = 0
for i, val in enumerate(truth):
    if low_RF[i] <= val <= up_RF[i]:
        correct += 1
print("Score RF: ", correct/len(truth))

In [ ]:
truth = y_test
correct = 0
for i, val in enumerate(truth):
    if low_GB[i] <= val <= up_GB[i]:
        correct += 1
print("Score RF: ", correct/len(truth))